In [42]:
from os import getcwd as os_getcwd
from os import chdir as os_chdir
from os import listdir as os_listdir
from os import makedirs as os_makedirs
from os.path import isdir as os_path_isdir
from os.path import exists as os_path_exists

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, ElementNotInteractableException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from pandas import DataFrame, read_html, concat, ExcelFile, ExcelWriter, Series
from pandas import isna as pd_isna
from pandas import merge as pd_merge
from math import isnan as math_isnan

from random import uniform as random_uniform
# from random import randint as random_randint
from time import sleep as time_sleep

from datetime import datetime as dt

from win32clipboard import OpenClipboard, EmptyClipboard, SetClipboardText, CF_UNICODETEXT, CloseClipboard
from pyautogui import hotkey as pyt_hotkey

from re import findall as re_findall
from re import search as re_search

from tqdm.notebook import tqdm

from shutil import copyfile as shutil_copyfile

In [43]:
# подгружаем собственные модули
home_dir = os_getcwd()
# необходимо поменять на рабочую папку со скриптами!
os_chdir(r'C:\Users\tabakaev_mv\Desktop\РАБОТА\РА\2023\ПОДГРУЗКА ЭЗ\scripts')
#print(os_getcwd())
from create_and_save import create_dir_save_table
from excel_parser import excel_to_data_frame_parser, printDimensionsOfDF
from check_dates import DateChecker
os_chdir(home_dir)
#print(os_getcwd())
print('Модули успешно подгружены!')

Модули успешно подгружены!


ФУНКЦИИ!!!

In [44]:
def random_sleep(upper_bound:int, lower_bound=0):
    random_digit_parse = random_uniform(lower_bound, upper_bound)
    time_sleep(random_digit_parse)

In [45]:
def get_browser(browser_full_path: str,
                driver_full_path: str,
                wait_secs: int=10):
    # готовый к работе веб-драйвер возращается методом класса
    options = webdriver.ChromeOptions()
    ###
    options.add_argument(r"--user-data=C:\Users\tabakaev_mv\AppData\Local\Yandex\YandexBrowser\User Data\Default")
    ###
    options.binary_location = browser_full_path
    options.add_argument("--disable-notifications")
    driver = webdriver.Chrome(executable_path=driver_full_path,
                              options=options)
    driver.implicitly_wait(wait_secs)
    return driver

In [46]:
def get_login_password(txt_file_path:str):
    login = ''
    password = ''
    with open(txt_file_path) as file:
        lines = [line.rstrip() for line in file]
    login = lines[0]
    password = lines[1]
    return(login, password)

In [47]:
def get_element_value(driver, xpath, timeout = 15):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
        return
    found_element = driver.find_element(By.XPATH, xpath)
    return(found_element.get_attribute('value'))

In [48]:
def get_element_title(driver, xpath, timeout = 15):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
        return
    found_element = driver.find_element(By.XPATH, xpath)
    return(found_element.get_attribute('title'))

In [49]:
def send_keys_by_xpath(driver, xpath, text, pause_secs=2.1, timeout=15, need_press_enter:bool=False):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
        return
    found_element = driver.find_element(By.XPATH, xpath)
    try:
        found_element.send_keys(text)
        if need_press_enter:
            random_sleep(upper_bound=pause_secs)
            found_element.send_keys(Keys.ENTER)
    except ElementNotInteractableException:
        print(f'Невозможно отправить текст: <{text}>')
        print(f'В форму или элемент: <{found_element}>')
        clipboard_copy(text=str(text), paste_value=False)
        input('Исправьте, пожалуйста, в ручную, а затем нажмите Enter для продолжения ввода информации по ЭИ!')

In [50]:
def click_element_by_xpath(driver, xpath, timeout = 15, in_new_window:bool=False):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
        return
    try:
        found_element = driver.find_element(By.XPATH, xpath)
        if in_new_window:
            found_element.click()
            # href = driver.execute_script("return arguments[0].outerHTML;", found_element)
            # print(href)
            # driver.execute_script(f'window.open({href}, "_blank1");')
        else:
            found_element.click()
        return(True)
    except ElementClickInterceptedException:
        print(f'<{found_element}>\nis not clickable at the moment!')
        print(f'Element XPath:\n<{xpath}>')
#         input('Для пропуска нажмите Enter!')

In [51]:
def close_browser(driver):
    input('После нажатия клавиши браузер будет закрыт!')
    try:
        driver.close()
        print('Инициализация закрытия Selenium-браузера прошла успешно!')
    except:
        print('Ошибка закрытия Selenium-браузера')

In [52]:
def find_digits_after_text(driver, text_mark: str):
    
    html_source = driver.page_source

    digit = 0
    
    if text_mark in html_source:
        print(f'Искомый элемент <{text_mark}> ПРИСУТСТВУЕТ на странице!')
        match = re_search(f'{text_mark}(\d+)', html_source)
        if match:
            digit = int(match.group(1))
            print(f'Найдено число: <{digit}>')
            return(digit, True)
    else:
        print(f'Искомый элемент <{text_mark}> ОТСУТСТВУЕТ на странице!')
        return(digit, False)

In [53]:
def delete_nan_rows(df: DataFrame):
    # НАЧАЛО ФУНКЦИИ ---//---
    # НЕОБХОДИМО УЗНАТЬ КОЛИЧЕСТВО СТРОК СО ВСЕМИ ПРОПУЩЕННЫМИ ЗНАЧЕНИЯМИ (по всем колонкам)!!!
    idx = list(df.index[df.isnull().all(1)])
    print(idx)
    if idx:
        # кол-во удаляемых строк, а не одна строка!!!
        print(f'Необходимо удалить <{len(idx)}> строк ДФ!')
    print()
    print(f'Размерность ДФ ДО УДАЛЕНИЯ i строк: <{df.shape}>')
    print()
    # reset index???
    df.drop(idx,inplace=True)
    df.reset_index(drop=True, inplace=True)
    print(f'Размерность ДФ ПОСЛЕ УДАЛЕНИЯ i строк: <{df.shape}>')
    return(df)
    # КОНЕЦ ФУНКЦИИ ---//---

In [ ]:
def delete_df_rows_values(df: DataFrame, colname: str, value):
    # НАЧАЛО ФУНКЦИИ ---//---
    # НЕОБХОДИМО УЗНАТЬ КОЛИЧЕСТВО СТРОК СО ВСЕМИ ПРОПУЩЕННЫМИ ЗНАЧЕНИЯМИ (по всем колонкам)!!!
    idx = list(df.index[df[colname] == value])
    print(idx)
    if idx:
        # кол-во удаляемых строк
        del_num_rows = len(idx)
        print(f'Необходимо удалить <{del_num_rows}> строк ДФ!')
        print()
        print(f'Размерность ДФ ДО УДАЛЕНИЯ i строк: <{df.shape}>')
        print()
        df.drop(idx,inplace=True)
        df.reset_index(drop=True, inplace=True)
        print(f'Размерность ДФ ПОСЛЕ УДАЛЕНИЯ <{del_num_rows}> строк: <{df.shape}>')
    return(df)

In [54]:
def parse_part_df(driver,
                  df_page_index:int=1,
                  df_colnames_page_index:int=0,
                  need_delete_nans:bool=True):
    # парсим таблицу со всем перечнем заявлений на получение ЭЛМК!!!
    # в переключателе выводить макисмальное количество строк по ЭЛМК!!!
    # подождать пока все строки прогрузятся!!!
    page_content_list = read_html(driver.page_source)
    df = DataFrame(page_content_list[df_page_index])
    # после аварийной перезагрузки сервером страницы
    # попытаемся выяснить тот же ДФ с заявлениями мы пытаемся спарсить или нет? 02.11.2024
    print(df.shape)
    df_colnames = list(DataFrame(page_content_list[df_colnames_page_index]))
    df.columns = df_colnames
    if need_delete_nans:
        df = delete_nan_rows(df=df)
    return(df, df.shape[0])

In [55]:
def send_command_2element(driver, xpath, timeout = 15, command:str='page_down'):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
        return
    found_element = driver.find_element(By.XPATH, xpath)
    if command == 'page_down':
        found_element.send_keys(Keys.PAGE_DOWN)

In [56]:
def move_2web_element(driver,
                      xpath,
                      timeout = 3,
                      need_click:bool=False,
                      in_new_window: bool=False):
    # driver.refresh()
    # другой способ скроллинга сраницы попробовать!
    # сверять после каждой итерации количество записей на странице,
    # если оно не изменилось, пробуем скроллить еще!!!
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
        return
    found_element = driver.find_element(By.XPATH, xpath)
    
    # прокручиваем страницу с помощью скрипта
    driver.execute_script("return arguments[0].scrollIntoView(true);", found_element)
    random_sleep(upper_bound=1.0, lower_bound=0)
    
    action = ActionChains(driver)
    if need_click:
        if in_new_window:
            action.move_to_element(found_element).perform()
            found_element.send_keys(Keys.COMMAND + 't')
        else:
            action.move_to_element(found_element).click().perform()
    else:
        action.move_to_element(found_element).perform()

In [57]:
def need_end_procedure(text_in):
    end_procedure = False
    text_in = text_in.lower().strip()
    if 'x' in text_in or 'х' in text_in:
        end_procedure = True
    return(end_procedure)

In [ ]:
def get_total_df(df, appln_number_colname, ERROR_APPL_NUMBER, driver):
    # удаляем пропущенные значения из общего ДФ !!!
    df, rows_df = parse_part_df(driver=driver,
                                need_delete_nans=True)
    # переводим номер заявления в целочисленное представление
    # df[appln_number_colname] = df[appln_number_colname].apply(lambda x: int(x))
    # ИСПРАВЛЕНИЕ 06.09.2024
    appl_numbers_lst = []
    for number in df[appln_number_colname].to_list():
        try:
            number_converted = int(number)
        # prevent ValueError: cannot convert float NaN to integer
        except ValueError:
            number_converted = ERROR_APPL_NUMBER
        appl_numbers_lst.append(number_converted)
    df[appln_number_colname] = appl_numbers_lst
    # DELETE ERRORS APPLS NUMBERS VALUES
    df = delete_df_rows_values(df, appln_number_colname, ERROR_APPL_NUMBER)
    print('Колонка с номерами заявлений успешно приведена к целочисленному значению!')
    return df.copy()

In [1]:
def parse_total_df(driver,
                   num_rows_mark:str,
                   appln_number_colname:str,
                   no_yes:str,
                   excel_temp_filename: str,
                   sleep_secs_up_to:float=1.5,
                   counter_max_value:int=150,
                   ask_for_cancel_interval:int=50):
    """
    Парсит всю таблицу с переводом ее в pandas-ДФ
    Возвращает ДФ и номера заявлений (в виде листа)
    """
    ERROR_APPL_NUMBER = -666
    driver.refresh()
    time_sleep(7)
    num_rows, num_rows_is_available = find_digits_after_text(driver=driver,
                                                             text_mark=num_rows_mark)
    print(num_rows, num_rows_is_available)

    counter = 0
    rows_df = 0
    need_parse_page = True
    PARSING_ERROR = False
    df = DataFrame(dict())
    
    try:
        while need_parse_page and counter < counter_max_value:
            counter += 1
            df = get_total_df(df, appln_number_colname, ERROR_APPL_NUMBER, driver)
            if counter % ask_for_cancel_interval == 0:
                if need_end_procedure(text_in=input('для отмены процесса парсинга введите х')):
                    need_parse_page = False
            if rows_df >= num_rows:
                print(f'\nСпарсили ВСЕ строки ДФ <{rows_df}> в ДОЛЖНОМ количестве <{num_rows}>!')
                need_parse_page = False
            else:
                print(f'\nСпарсили НЕ ВСЕ строки ДФ <{rows_df}> в ДОЛЖНОМ количестве <{num_rows}>!')

                random_sleep(upper_bound=sleep_secs_up_to)
                df, rows_df = parse_part_df(driver=driver,
                                                      need_delete_nans=False)
                # ИСПРАВЛЕНИЕ ОТ 06.09.2024!!!
                # ПРОВЕРКА НА ПУСТОЕ ЗНАЧЕНИЕ № ЗАЯВЛЕНИЯ
                is_number_empty = True
                tries = 0
                while is_number_empty and tries < 15:
                    tries += 1
                    try:
                        locator_xpath = f"//*[contains(text(), '{int(df.iloc[df.shape[0]-tries,1])}')]"
                        is_number_empty = False
                    # ValueError: cannot convert float NaN to integer
                    except ValueError:
                        pass
                # скроллим страницу вниз по последнему отображенному номеру заявления в таблице!!!
                try:
                    move_2web_element(driver=driver, xpath=locator_xpath)
                except StaleElementReferenceException:
                    pass
    except BaseException as error:
        print('Caught this error: ' + repr(error))
        print(f'Парсинг ДФ на текущей странице завершен В АВАРИЙНОМ РЕЖИМЕ за <{counter}> циклов')
        PARSING_ERROR = True
    if PARSING_ERROR == False:
        print(f'Парсинг ДФ на текущей странице завершен В СТАНДАРТНОМ РЕЖИМЕ за <{counter}> циклов')
        df = get_total_df(df, appln_number_colname, ERROR_APPL_NUMBER, driver)
    appl_numbers = df[appln_number_colname].to_list()
    check_apl_numbs_duplicates = no_yes[len(appl_numbers) == len(set(appl_numbers))]
    # (06.09.2024)
    print(f'\nДУБЛИКАТЫ В ТАБЛИЦЕ ЗАЯВЛЕНИЙ ЭЛМК ОТСУТСТВУЮТ: <{check_apl_numbs_duplicates}>')
    
    df.to_excel(excel_temp_filename, index=False)
    
    return(df, appl_numbers)

In [59]:
def find_element_xpath(driver, xpath, timeout = 3):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath))
        WebDriverWait(driver, timeout).until(element_present)
        return(True)
    except TimeoutException:
        print("Timed out waiting for page to load")
        return(False)

In [60]:
def get_filial_name(first_lst_in: list,
                    second_lst_in: list,
                    na_value_out:str='Н/Д'):

    if len(first_lst_in) != len(second_lst_in):
        print('Длина поданных листов не совпадает!')
        return
    
    filial_list = []
    for ind in range(len(first_lst_in)):
        i = 0
        for lst in [second_lst_in, first_lst_in]:  # фактический адрес (2-ой лист) - важнее, чем регистрация (первый)! 
            i += 1
            row = lst[ind]
            #print(row)
            text = str(row).lower()
            text = text.strip()
            if i == 1 and 'кузбасс' not in text:
                continue # факт проживает - в другом регионе
            if len(filial_list) != ind:
                continue # филиал - уже найден на предыдущей итерации для этого адреса!
            if ('кемерово' in text  or
                'кемеровский' in text):
                filial_list.append('Кемерово')
            elif 'новокузнецк' in text:
                filial_list.append('Новокузнецк')
            #elif 'киселевск' in text:
            #    filial_list.append('Киселевск')
            elif ('березовский' in text or
                'топки' in text) and 'ленинск-' not in text:
                filial_list.append('Березовский')
            elif ('белов' in text or
                  'бачатск' in text or
                  'краснобродск' in text or
                  'грамотеин' in text):
                filial_list.append('Белово')
            elif ('прокопьевск' in text or
                 'киселевск' in text):
                filial_list.append('Прокопьевск')
            elif ('-кузнецкий' in text or
               'ленинск-кузн' in text or
               'полысаев' in text):
                filial_list.append('Л-Кузнецк')
            elif ('анжеро' in text or
               'судженск' in text or
               text.startswith('яя') or
               'яйский' in text or
               'ижморск' in text or
               'ижморка' in text):
                filial_list.append('А-Судженск')
            elif ('гурьевск' in text or
               'салаир' in text):
                filial_list.append('Гурьевск')
            elif ('осинник' in text or
               'калтан' in text):
                filial_list.append('Осинники')
            elif ('таштагол' in text or
               'шерегеш' in text):
                filial_list.append('Таштагол')
            elif ('междур' in text or
                  'межд.' in text or
               'мыск' in text):
                filial_list.append('Мыски')
            elif ('мариинск' in text or
               'чебул' in text or
               'чебулинск' in text or
               'тисуль' in text or
               'тяжин' in text):
                filial_list.append('Мариинск')
            elif ('юрга' in text or
               'юргинский' in text):
                filial_list.append('Юрга')
            elif ('крапивин' in text or
               'промышленн' in text):
                filial_list.append('Промышленная')
            elif ('яшкин' in text or
               'тайг' in text):
                filial_list.append('Яшкино')
        if i ==2 and len(filial_list) == ind:
            filial_list.append(na_value_out)

    print(len(filial_list))
    return(filial_list)

In [61]:
def append_timestamp_filename(filename_in:str,
                              delim:str='_',
                              file_type:str='.xlsx'):
    filename_out = ''
    t = DateChecker('')
    today_timestamp = t.get_today_date(date_format='medium')
    # print(today_timestamp)
    today_timestamp = today_timestamp.replace(' ', delim).replace(':', delim).replace('.', delim)
    filename_out = filename_in + delim + today_timestamp + file_type
    return(filename_out)

ПЕРЕМЕННЫЕ ГЛОБАЛЬНОГО ОКРУЖЕНИЯ!!!

In [62]:
BROWSER_FILE_PATH = "C:\\Program Files\\Yandex\\YandexBrowser\\Application\\browser.exe"
WEBDRIVER_PATH = "C:\\yandexdriver\\yandexdriver.exe"
HOME_URL = "https://eias.rospotrebnadzor.ru/"
ELMK_URL = "https://eias.rospotrebnadzor.ru/gov-services/elmks"
LOGIN_PASSWORD_FULLPATH = "C:\\Users\\tabakaev_mv\\Desktop\\РАБОТА\\ЕИАС\\2023\\ТОКСИКОМОНИТОРИНГ\\ПОДГРУЗКА\\application\\eias_pass.txt"
UPLOAD_LAN_DIR = "\\192.168.201.38\\общая папка\\06 ООДЦ\\Выгрузка_ЭЛМК\\"
# получаем логин и пароль для входа
LOGIN, PASSWORD = get_login_password(txt_file_path=LOGIN_PASSWORD_FULLPATH)
NUM_ROWS_MARK = 'Количество записей: '
TEMP_XLSX_FILENAME = "temp_xlsx_elmk_first_df.xlsx"

# выводы(print-ы) о проверках
NO_or_YES = ["НЕТ", "ДА"]
JOIN_WARNING = [
    "FULL INNER JOIN IS NOT COMPLETED!!!",
    "FULL INNER JOIN IS COMPLETED!!!"
]

# Наименования колонок в ИСХОДНОМ ДФ
APPLN_NUMBER_COLNAME = '№ заявления'

# НАИМЕНОВАНИЯ ПОЛЕЙ В СЛОВАРЕ СЛОВАРЕЙ
WAIT_DICT = "для_служебн_польз"
REGISTR_ADRESS_DICT = 'адрес_регистрации'
REGISTR_ADRESS_BUILDING_DICT = 'рег_ном_здания'
REGISTR_ADRESS_FLAT_DICT = 'рег_ном_квартиры'
FACT_ADRESS_DICT = 'адрес_фактический'
GENDER_DICT = 'пол'
TEL_NUMB_DICT = 'номер_телефона'
LAST_NAME_DICT = 'фамилия'
FIRST_NAME_DICT = 'имя'
MIDDLE_NAME_DICT = 'отчество'
EMAIL_DICT = 'е_майл'
PASSPRT_NUMB_DICT = 'паспорт_номер'
PASSPRT_DATE_DICT = 'паспорт_дата_выдачи'
PASSPRT_DEPT_DICT = 'паспорт_кем_выдан'
PASSPRT_CODE_DICT = 'паспорт_код_подразделения'

# в объединенном ДФ
FILIAL_COLNAME = 'филиал'

COLNAMES_DICT = (
    WAIT_DICT,
    REGISTR_ADRESS_DICT,
    # REGISTR_ADRESS_BUILDING_DICT,
    # REGISTR_ADRESS_FLAT_DICT,
    FACT_ADRESS_DICT,
    GENDER_DICT,
    TEL_NUMB_DICT,
    LAST_NAME_DICT,
    FIRST_NAME_DICT,
    MIDDLE_NAME_DICT,
    EMAIL_DICT,
    PASSPRT_NUMB_DICT,
    PASSPRT_DATE_DICT,
    PASSPRT_DEPT_DICT,
    PASSPRT_CODE_DICT)

# XPATHS!!!
LOGIN_XPATH = "/html/body/main/div/form/div[1]/input"
PASSWORD_XPATH = "/html/body/main/div/form/div[2]/input"
SUBMIT_PASS_XPATH = "/html/body/main/div/form/button"
#---!!! personal data !!! ---# beginning
WAIT_XPATH = ""
# REGISTR_ADRESS_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/app-address/div/div/div[1]/nz-form-item/nz-form-control/div/div/nz-input-group/input"
REGISTR_ADRESS_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/div[7]/div/nz-form-item/nz-form-control/div/div/input"
REGISTR_ADRESS_BUILDING_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/app-address/div/div/div[2]/nz-form-item/nz-form-control/div/div/nz-select/nz-select-top-control/nz-select-item"
REGISTR_ADRESS_FLAT_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/app-address/div/div/div[3]/nz-form-item/nz-form-control/div/div/nz-input-group/input"
FACT_ADRESS_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/div[7]/app-address/div/div/div[1]/nz-form-item/nz-form-control/div/div/nz-input-group/input"
FACT_ADRESS_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/div[9]/div[2]/div/nz-form-item/nz-form-control/div/div/input"
GENDER_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/div[3]/div[4]/nz-form-item/nz-form-control/div/div/app-select/div/nz-select/nz-select-top-control/nz-select-item"
TEL_NUMB_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/div[3]/div[7]/nz-form-item/nz-form-control/div/div/input"
LAST_NAME_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/div[3]/div[1]/nz-form-item/nz-form-control/div/div/input"
FIRST_NAME_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/div[3]/div[2]/nz-form-item/nz-form-control/div/div/input"
MIDDLE_NAME_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/div[3]/div[3]/nz-form-item/nz-form-control/div/div/input"
EMAIL_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/div[3]/div[8]/nz-form-item/nz-form-control/div/div/input"
PASSPRT_NUMB_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/div[4]/div[1]/nz-form-item/nz-form-control/div/div/input"
PASSPRT_DATE_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/div[4]/div[2]/nz-form-item/nz-form-control/div/div/nz-date-picker/div/input"
PASSPRT_DEPT_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/div[4]/div[3]/nz-form-item/nz-form-control/div/div/input"
PASSPRT_CODE_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmk/div/div[3]/form/div[2]/div[4]/div[4]/nz-form-item/nz-form-control/div/div/input"

PERS_DATA_XPATH = {
    WAIT_XPATH:[0,0],
    REGISTR_ADRESS_XPATH:[0,0],
    # REGISTR_ADRESS_BUILDING_XPATH:[1,0],
    # REGISTR_ADRESS_FLAT_XPATH:[0,0],
    FACT_ADRESS_XPATH:[0,1],
    GENDER_XPATH:[1,0],
    TEL_NUMB_XPATH:[0,0],
    LAST_NAME_XPATH:[0,0],
    FIRST_NAME_XPATH:[0,0],
    MIDDLE_NAME_XPATH:[0,0],
    EMAIL_XPATH:[0,0],
    PASSPRT_NUMB_XPATH:[0,0],
    PASSPRT_DATE_XPATH:[0,0],
    PASSPRT_DEPT_XPATH:[0,0],
    PASSPRT_CODE_XPATH:[0,0]}

#---!!! personal data !!! ---# ends
DF_ROW_NUMS_XPATH = "//*[contains(text(), 'Количество записей: ')]"
DF_TABLE_SCROLL_XPATH = "/html/body/app-root/nz-layout/nz-layout/nz-layout/div/app-elmks/div/div[3]/app-table/nz-table/nz-spin/div/div/nz-table-inner-scroll/div[2]"

САМ КОД!!!

In [63]:
browser = get_browser(browser_full_path=BROWSER_FILE_PATH,
                      driver_full_path=WEBDRIVER_PATH)
# домашняя страница портала
browser.get(url=HOME_URL)
input('Выберите и подтвердите сертификат, а затем нажмите клавишу Enter!!!')

Выберите и подтвердите сертификат, а затем нажмите клавишу Enter!!!


''

In [64]:
# ИСПРАВИТЬ!!!
# ввод логина и пароля для входа на портал!
send_keys_by_xpath(driver=browser,
                   xpath=LOGIN_XPATH,
                   text=LOGIN,
                   pause_secs=2.1,
                   timeout=15,
                   need_press_enter=False)

send_keys_by_xpath(driver=browser,
                   xpath=PASSWORD_XPATH,
                   text=PASSWORD,
                   pause_secs=2.1,
                   timeout=15,
                   need_press_enter=True)
# input('Подождать пока страница прогрузится?')
# переходим на страницу ЕЛМК
browser.get(url=ELMK_URL)
# подождать (?) пока контент прогрузится...

ЦИКЛ ПО СТРАНИЦАМ?!

In [65]:
answer = input("Загрузить ранее выгруженный ДФ: да (y) / нет (n)?").strip().lower()
if "y" in answer or "да" in answer:
    appl_df = excel_to_data_frame_parser(file=TEMP_XLSX_FILENAME,
                               sheet_name="Sheet1",
                               rows_to_skip=0,
                               blank_values_drop=0,
                               first_row_header=0)
    printDimensionsOfDF(dfInput=appl_df,
                        warnStr="подгрузки временного ранее сохраненного ДФ")
    appl_numbers = appl_df[APPLN_NUMBER_COLNAME].to_list()
else:
    appl_df, appl_numbers = parse_total_df(driver=browser,
                                           num_rows_mark=NUM_ROWS_MARK,
                                           appln_number_colname=APPLN_NUMBER_COLNAME,
                                           no_yes=NO_or_YES,
                                           excel_temp_filename=TEMP_XLSX_FILENAME,
                                           sleep_secs_up_to=1.2,
                                           counter_max_value=350,
                                           ask_for_cancel_interval=100)

Искомый элемент <Количество записей: > ПРИСУТСТВУЕТ на странице!
Найдено число: <196>
196 True

Спарсили НЕ ВСЕ строки ДФ <0> в ДОЛЖНОМ количестве <196>!

Спарсили НЕ ВСЕ строки ДФ <21> в ДОЛЖНОМ количестве <196>!

Спарсили НЕ ВСЕ строки ДФ <21> в ДОЛЖНОМ количестве <196>!

Спарсили НЕ ВСЕ строки ДФ <41> в ДОЛЖНОМ количестве <196>!

Спарсили НЕ ВСЕ строки ДФ <41> в ДОЛЖНОМ количестве <196>!

Спарсили НЕ ВСЕ строки ДФ <41> в ДОЛЖНОМ количестве <196>!

Спарсили НЕ ВСЕ строки ДФ <61> в ДОЛЖНОМ количестве <196>!

Спарсили НЕ ВСЕ строки ДФ <81> в ДОЛЖНОМ количестве <196>!

Спарсили НЕ ВСЕ строки ДФ <101> в ДОЛЖНОМ количестве <196>!

Спарсили НЕ ВСЕ строки ДФ <121> в ДОЛЖНОМ количестве <196>!

Спарсили НЕ ВСЕ строки ДФ <141> в ДОЛЖНОМ количестве <196>!

Спарсили НЕ ВСЕ строки ДФ <161> в ДОЛЖНОМ количестве <196>!

Спарсили НЕ ВСЕ строки ДФ <181> в ДОЛЖНОМ количестве <196>!

Спарсили ВСЕ строки ДФ <197> в ДОЛЖНОМ количестве <196>!
Парсинг ДФ на текущей странице завершен за <14> циклов
[0]
Необ

ПАРСИНГ ДАННЫХ ИЗ КАРТОЧКИ КАЖДОГО ЗАЯВЛЕНИЯ НА ТЕКУЩЕЙ СТРАНИЦЕ

In [66]:
# for colname, xpath in zip(COLNAMES_DICT, PERS_DATA_XPATH):
#    print(colname, xpath)
# [i for i in PERS_DATA_XPATH]

In [67]:
def get_personal_data(driver,
                      sleep_up_to:float,
                      in_new_window: bool=False):
    # открываем в новом окне и закрываем после парсинга содержимого
    # if in_new_window:
    #    pass
    # driver.execute_script('window.open("http://parsinger.ru/blank/2/2.html", "_blank1");')
    if in_new_window:
        if len(list(driver.window_handles))>1:
            # Переключаемся на новую (открытую) вкладку
            driver.switch_to.window(driver.window_handles[1])
    out_dict = {}
    global COLNAMES_DICT, PERS_DATA_XPATH
    for colname, xpath in zip(COLNAMES_DICT, PERS_DATA_XPATH):
        # print(colname, xpath)
        curr_value = ''
        need_get_title = PERS_DATA_XPATH[xpath][0]
        need_fast_check = PERS_DATA_XPATH[xpath][1]
        
        if need_fast_check:
            element_is_available = find_element_xpath(driver=driver,
                                                  xpath=xpath,
                                                  timeout = 1)
        else:
            element_is_available = find_element_xpath(driver=driver,
                                                  xpath=xpath,
                                                  timeout = 3)
            
        if element_is_available and not(need_get_title):
            curr_value = get_element_value(driver=driver,
                                        xpath=xpath,
                                        timeout = 15)
        elif need_get_title:
            curr_value = get_element_title(driver=driver,
                                        xpath=xpath,
                                        timeout = 15)
        out_dict[colname] = curr_value
        random_sleep(upper_bound=sleep_up_to)
    if in_new_window:
        if len(list(driver.window_handles))>1:
            driver.close()
            # Переключаемся на исходную вкладку
            driver.switch_to.window(driver.window_handles[0])
        else:
            driver.back()
        # driver.close()
    else:
        driver.back()
    return(out_dict)

In [68]:
%%time
ask_for_cancel_interval = 5000 # периодичность по количеству заявлений - запрос на завершение процедуры
sleep_secs_up_to = 1.1
sleep_secs_up_to_pesr_data = 0.5
element_found = False
browser.refresh()

appl_dict = {}
counter = 0
stop_parsing = False
for number in tqdm(appl_numbers):
    # переименовать флаг!!!
    if stop_parsing: # завершаем досрочно, если юзер ввел х (русс / англ. раскладка)
        break
    need_parse_appl = True
    tries = 0
    while need_parse_appl:
        tries += 1
        if abs(tries) % 5 == 0:
            print(f"Trying to refresh homepage <{ELMK_URL}> on try number <{tries}>")
            browser.get(url=ELMK_URL)
        if counter % ask_for_cancel_interval == 0:
            if need_end_procedure(text_in=input('для отмены процесса парсинга введите х')):
                print('Процесс - прерван ...')
                stop_parsing = True
                need_parse_appl = False
                break
        print(number)
        counter += 1
        try:
            appl_xpath = f"//*[contains(text(), '{number}')]"
            random_sleep(upper_bound=sleep_secs_up_to)
            element_found = find_element_xpath(driver=browser,
                                                xpath=appl_xpath)
            try:
                if element_found:
                    pass
                else:  # ссылка (по XPath) на заявление не найдена!
                    # print(f'Пропуск парсинга по номеру: <{number}>')
                    # весь код после слова "continue" - не будет выполнен на данной итерации!
                    need_scroll_down = True
                    while need_scroll_down:
                        df_temp, rows_df = parse_part_df(driver=browser,
                                                         need_delete_nans=False)
                        locator_xpath = f"//*[contains(text(), '{int(df_temp.iloc[df_temp.shape[0]-1,1])}')]"
                        # скроллим страницу вниз по последнему отображенному номеру заявления в таблице!!!
                        move_2web_element(driver=browser, xpath=locator_xpath)
                        element_found = find_element_xpath(driver=browser,
                                                    xpath=appl_xpath)
                        if element_found:
                            need_scroll_down = False
                    # continue
            except IndexError:
                random_sleep(upper_bound=15, lower_bound=5)
            except ValueError:
                browser.refresh()
                random_sleep(upper_bound=10, lower_bound=3)
                browser.get(url=ELMK_URL)
                random_sleep(upper_bound=5, lower_bound=3)
            
            click_element_by_xpath(driver=browser,
                                   xpath=appl_xpath,
                                   timeout=15,
                                   in_new_window=True)
            need_parse_appl = False
            
        except StaleElementReferenceException:
            pass
    # ПОЛУЧАЕМ ПЕРСОНАЛЬНЫЕ ДАННЫЕ!
    if not(stop_parsing):
        appl_dict[number] = get_personal_data(driver=browser,
                                              sleep_up_to=sleep_secs_up_to_pesr_data,
                                              in_new_window=True)
        

print(f'\nКоличество спарсенных строк таблицы заявлений составило: <{len(appl_dict)}>')

для отмены процесса парсинга введите х
3582525456
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
3582279346
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
3574052537
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
3570428940
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
3569492973
Timed out waiting for page to l

Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
3550517910
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
3549119135
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
3547883863
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
3541222807
Timed out w

Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
3519695244
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
3516522183
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for page to load
3518045447
Timed out waiting for page to load
Timed out waiting for page to load
Timed out waiting for 

ValueError: No tables found

In [69]:
# appl_df.head()

In [70]:
# словарь словарей - в ДФ
# номер заявления переносим в индекс!
df_pers_data = DataFrame.from_dict(appl_dict).T.reset_index()
df_pers_data.rename({'index': APPLN_NUMBER_COLNAME}, axis='columns', inplace=True)
# df_pers_data.head()

МЕРДЖИМ ДВА ДФ ПО НОМЕРУ ЗАЯВЛЕНИЯ!!!

In [71]:
df_merged = pd_merge(appl_df,
                     df_pers_data,
                     on=APPLN_NUMBER_COLNAME,
                     how='left')
rows_equal = appl_df.shape[0] == df_pers_data.shape[0]
join_warn = JOIN_WARNING[rows_equal]
print(join_warn)
if not (rows_equal):
    open_table_name = 'ОТКРЫТАЯ ТАБЛИЦА'
    personal_data_table = 'ТАБЛИЦА по ПЕРСОНАЛЬНЫМ ДАННЫМ (скрытая)'
    print(f'\nРазмерность ДФ из <{open_table_name}> составила: <{appl_df.shape}>')
    print(f'\nРазмерность ДФ, из <{personal_data_table}> составила: <{df_pers_data.shape}>')
    print('\nПо всей видимости не доПарсено следующее количество строк')
    print(f'<{abs(appl_df.shape[0] - df_pers_data.shape[0])}> из ДФ:')
    print([
        open_table_name,
        personal_data_table
    ][appl_df.shape[0] > df_pers_data.shape[0]])
    print()
print(df_merged.shape)

# заменить пропущенные значения на пустые строки
df_merged.fillna('', inplace=True)

FULL INNER JOIN IS NOT COMPLETED!!!

Размерность ДФ из <ОТКРЫТАЯ ТАБЛИЦА> составила: <(196, 13)>

Размерность ДФ, из <ТАБЛИЦА по ПЕРСОНАЛЬНЫМ ДАННЫМ (скрытая)> составила: <(69, 16)>

По всей видимости не доПарсено следующее количество строк
<127> из ДФ:
ТАБЛИЦА по ПЕРСОНАЛЬНЫМ ДАННЫМ (скрытая)

(196, 28)


In [72]:
# df_merged.head(30)

ОПРЕДЕЛЕНИЕ ФИЛИАЛА ПО АДРЕСУ РЕГИСТРАЦИИ ЗАЯВИТЕЛЯ!

In [73]:
df_merged[FILIAL_COLNAME] = get_filial_name(first_lst_in=df_merged[REGISTR_ADRESS_DICT].to_list(),
                                            second_lst_in=df_merged[FACT_ADRESS_DICT].to_list())

196


In [ ]:
pc_file_name = append_timestamp_filename(filename_in='merged_ELMK')
lan_file_name = append_timestamp_filename(filename_in=UPLOAD_LAN_DIR+'merged_ELMK')

In [ ]:
df_merged.to_excel(pc_file_name, index=False)

In [ ]:
shutil_copyfile(pc_file_name, lan_file_name)
if os_path_exists(lan_file_name):
    print(f'DF in excel-file format has been successfully copied into <{UPLOAD_LAN_DIR}> dir!')
    print(':-)')
else:
    print('ATTENTION!!!')
    print(f'DF in excel-file format has NOT been successfully copied into <{UPLOAD_LAN_DIR}> dir!')
    print(':,-(((')

In [75]:
input('Программа завершена без ошибок! ;-)')
close_browser(driver=browser)

Программа завершена без ошибок! ;-)и
После нажатия клавиши браузер будет закрыт!
Инициализация закрытия Selenium-браузера прошла успешно!


In [35]:
# df_merged[REGISTR_ADRESS_DICT].head(20)

In [36]:
# df_merged.head(30)

ЧЕРНОВИКИ!!!

In [37]:
# row_nums_max_text = get_element_value(driver=browser,
#                                      xpath=DF_ROW_NUMS_XPATH,
#                                      timeout = 15)
# print(row_nums_max_text)

In [38]:
# получаем адрес регистрации
# curr_registr_adress = get_element_value(driver=browser,
#                      xpath=REGISTR_ADRESS_XPATH,
#                      timeout = 15)
#print(curr_registr_adress)

In [39]:
# ask_for_cancel_interval = 50
# sleep_secs_up_to = 1.1
# appl_xpath_interval = 20
# appl_xpath_bounds = []

# appl_dict = {}
# counter = 0
# for number in tqdm(appl_numbers):
#     print(number)
#     counter += 1
#     appl_xpath = f"//*[contains(text(), '{number}')]"
#     random_sleep(upper_bound=sleep_secs_up_to)
#     if (counter >= appl_xpath_interval or 
#     click_element_by_xpath(driver=browser,
#                                xpath=appl_xpath,
#                                timeout=15)):
#         pass
#     else:
#         for prev_appl_xpath in appl_xpath_bounds:
#             move_2web_element(driver=browser, xpath=prev_appl_xpath)
#             random_sleep(upper_bound=sleep_secs_up_to)
#         click_element_by_xpath(driver=browser,
#                                xpath=appl_xpath,
#                                timeout=15)
#     if counter % appl_xpath_interval == 0:
#         appl_xpath_bounds.append(appl_xpath)
    
#     # получаем адрес регистрации
#     curr_registr_adress = get_element_value(driver=browser,
#                           xpath=REGISTR_ADRESS_XPATH,
#                           timeout = 15)
#     #print(curr_registr_adress)
#     appl_dict[number] = {}
#     appl_dict[number][REGISTR_ADRESS_DICT] = curr_registr_adress
#     if counter % ask_for_cancel_interval == 0:
#             if need_end_procedure(text_in=input('для отмены процесса парсинга введите х')):
#                 print('Процесс - прерван ...')
#                 break
#     random_sleep(upper_bound=sleep_secs_up_to)
#     browser.back()
# print(f'\nКоличество спарсенных строк таблицы заявлений составило: <{len(appl_dict)}>')

In [40]:
# адрес регистрации
# curr_registr_adress = get_element_value(driver=browser,
#                      xpath=REGISTR_ADRESS_XPATH,
#                      timeout = 15)
# appl_dict[number][REGISTR_ADRESS_DICT] = curr_registr_adress
# random_sleep(upper_bound=sleep_secs_up_to_pesr_data)

# DataFrame.from_dict(appl_dict).T.index.to_list()

In [41]:
# test_filename = append_timestamp_filename(filename_in='')
# print(test_filename)

In [74]:
# df_merged.to_excel(append_timestamp_filename(filename_in=UPLOAD_LAN_DIR+'merged_ELMK'), index=False)